In [1]:
from IPython.display import YouTubeVideo
YouTubeVideo('FPgo-hI7OiE')

# 如何使用和开发微信聊天机器人的系列教程
# A workshop to develop & use an intelligent and interactive chat-bot in WeChat

### WeChat is a popular social media app, which has more than 800 million monthly active users.

<img src='http://www.kudosdata.com/wp-content/uploads/2016/11/cropped-KudosLogo1.png' width=30% style="float: right;">
<img src='reference/WeChat_SamGu_QR.png' width=10% style="float: right;">

### http://www.KudosData.com

by: Sam.Gu@KudosData.com


May 2017 ========== Scan the QR code to become trainer's friend in WeChat ========>>

### 第五课：视频识别和处理
### Lesson 5: Video Recognition & Processing
* 识别视频消息中的物体名字 (Recognize objects in video)
* 识别视频的场景 (Detect scenery in video)
* 直接搜索视频内容 (Search content in video)

### Using Google Cloud Platform's Machine Learning APIs

From the same API console, choose "Dashboard" on the left-hand menu and "Enable API".

Enable the following APIs for your project (search for them) if they are not already enabled:
<ol>
<li> Google Cloud Vision API </li>
<li> Google Cloud Speech API </li>
<li> Google Cloud Translate API </li>
<li> Google Cloud Natural Language API </li>
<li> Google Cloud Video API </li>
</ol>

Finally, because we are calling the APIs from Python (clients in many other languages are available), let's install the Python package (it's not installed by default on Datalab)

In [2]:
# Copyright 2016 Google Inc.
# Licensed under the Apache License, Version 2.0 (the "License"); 
# !pip install --upgrade google
# !pip install --upgrade google-api-python-client
# !pip install --upgrade google.appengine

### 导入需要用到的一些功能程序库：

In [3]:
import io, os, subprocess, sys, time, datetime, requests, itchat
from itchat.content import *
from googleapiclient.discovery import build

In [4]:
import json, numpy as np
# Python 2
if sys.version_info[0] < 3:
    import urllib2
# Python 3
else:
    import urllib.request

### Using Google Cloud Platform's Machine Learning APIs

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Copy-paste your API Key here:

In [5]:
# Here I read in my own API_KEY from a file, which is not shared in Github repository:
with io.open('../../../API_KEY.txt') as fp: 
    for line in fp: APIKEY = line

# You need to un-comment below line and replace 'APIKEY' variable with your own GCP API key:
# APIKEY='AIzaSyCvxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [6]:
# Below is for GCP Video API
# video_service = build('videointelligence', 'v1', developerKey=APIKEY)
video_service = build('videointelligence', 'v1beta1', developerKey=APIKEY)
# check video processing progress
video_operation_service = build('videointelligence', 'v1', developerKey=APIKEY)

### 图片二进制base64码转换 (Define image pre-processing functions)

In [7]:
# Import the base64 encoding library.
import base64
# Pass the media data to an encoding function.
def encode_media(media_file):
    with io.open(media_file, "rb") as media_file:
        media_content = media_file.read()
# Python 2
    if sys.version_info[0] < 3:
        return base64.b64encode(media_content)
# Python 3
    else:
        return base64.b64encode(media_content).decode('utf-8')


### 机器智能API接口控制参数 (Define control parameters for API)

In [8]:
# API control parameter for 视频识别和处理 Video Recognition & Processing
parm_video_api_features = ['LABEL_DETECTION', 'SHOT_CHANGE_DETECTION']
# parm_video_api_features = ['LABEL_DETECTION']
# parm_video_api_features = ['SHOT_CHANGE_DETECTION']

# WIP 1

In [9]:
# Running Video API
def KudosData_VIDEO_DETECTION(video_base64):
##########################################################################################
# 1. Send request for video processing
##########################################################################################
    request1 = video_service.videos().annotate(body={
#               "inputUri": string,
              "inputContent": video_base64,
              "features": parm_video_api_features, # 'LABEL_DETECTION' & 'SHOT_CHANGE_DETECTION'
        })
    responses1 = request1.execute(num_retries=3)
    
    if 'name' in responses1: 
        print ('---------------------------------------')
        print ('Internal video name assinged by Google: %s' % responses1['name'])
##########################################################################################
# 2. Check progress till completion (Video requires asynchronous long processing...)
##########################################################################################
        responses2 = KudosData_VIDEO_DETECTION_CHECK_PROCRESS(responses1['name'], APIKEY)
    else:
        print ('[ERROR] Calling Video API request failed. Please re-try.')

##########################################################################################
# 3. Format results nicely
##########################################################################################
    responses3 = responses2
    
#     video_analysis_reply = u'\n[ Video Analysis 视频分析结果 ]'

#     video_analysis_reply += u'\n\n[ 视频分析 物体识别 ]\n'   
    
#     video_analysis_reply += u'\n\n[ 视频分析 物体识别 ]\n'
    
    return responses3

In [10]:
# Keep checking progress of Video API processing, till full completion.
def KudosData_VIDEO_DETECTION_CHECK_PROCRESS(name, apikey):
    flag_completion = False
    operation_url = 'https://videointelligence.googleapis.com/v1/operations/' + name + '?key=' + apikey
#     print('operation_url : ', operation_url)
    while not flag_completion:
#         print('... time.sleep')
        # Python 2
        if sys.version_info[0] < 3:
            response_operation = json.loads(urllib2.urlopen(operation_url).read())
        # Python 3
        else:
            response_operation = json.loads(urllib.request.urlopen(operation_url).read())
        
        # define an array, containing 'progress %' of each Video API
        video_api_progress_pct = np.zeros(shape=(1, len(response_operation['metadata']['annotationProgress'])))
#         print('... length : %d' % len(response_operation['metadata']['annotationProgress']))
        for i in range(len(response_operation['metadata']['annotationProgress'])):
            if 'progressPercent' in response_operation['metadata']['annotationProgress'][i]:
                video_api_progress_pct[0][i] = response_operation['metadata']['annotationProgress'][i]['progressPercent']
#                 print ('>>> Video API {} Progress : {} %'.format(i+1, video_api_progress_pct[0][i]))
        print ('--- Video API Overall Progress : {} %'.format(np.mean(video_api_progress_pct[0])))
        if min(video_api_progress_pct[0]) == 100: flag_completion = True
        time.sleep(1)

    return response_operation

In [11]:
# responses2 = KudosData_VIDEO_DETECTION_CHECK_PROCRESS('asia-east1.8208096652960631431', APIKEY)
# 3. Format results nicely
# responses2

In [12]:
# video_file = 'reference/SampleVideo_360x240_1mb.mp4'
video_file = 'reference/SampleVideo_360x240_2mb.mp4'

In [13]:
video_response = KudosData_VIDEO_DETECTION(encode_media(video_file))

---------------------------------------
Internal video name assinged by Google: asia-east1.109353954956704023
--- Video API Overall Progress : 0.0 %
--- Video API Overall Progress : 0.0 %
--- Video API Overall Progress : 50.0 %
--- Video API Overall Progress : 50.0 %
--- Video API Overall Progress : 50.0 %
--- Video API Overall Progress : 100.0 %


In [ ]:
# video_response

In [14]:
video_response['name']

u'asia-east1.109353954956704023'

In [15]:
video_response['done']

True

In [16]:
video_response['metadata']

{u'@type': u'type.googleapis.com/google.cloud.videointelligence.v1beta1.AnnotateVideoProgress',
 u'annotationProgress': [{u'progressPercent': 100,
   u'startTime': u'2017-06-05T08:24:59.269719Z',
   u'updateTime': u'2017-06-05T08:25:11.652105Z'},
  {u'progressPercent': 100,
   u'startTime': u'2017-06-05T08:24:59.269719Z',
   u'updateTime': u'2017-06-05T08:25:02.484499Z'}]}

In [17]:
# Version of Video API used:
video_response['response']['@type']

u'type.googleapis.com/google.cloud.videointelligence.v1beta1.AnnotateVideoResponse'

In [18]:
video_response['response']['annotationResults'][0]

{u'labelAnnotations': [{u'description': u'Animal',
   u'languageCode': u'en-us',
   u'locations': [{u'confidence': 0.8764819,
     u'level': u'VIDEO_LEVEL',
     u'segment': {u'endTimeOffset': u'-1', u'startTimeOffset': u'-1'}},
    {u'confidence': 0.89243287,
     u'level': u'SHOT_LEVEL',
     u'segment': {u'endTimeOffset': u'8333309'}},
    {u'confidence': 0.4363379,
     u'level': u'SHOT_LEVEL',
     u'segment': {u'endTimeOffset': u'12933314',
      u'startTimeOffset': u'8399998'}},
    {u'confidence': 0.7916033,
     u'level': u'SHOT_LEVEL',
     u'segment': {u'endTimeOffset': u'15999984',
      u'startTimeOffset': u'13000003'}},
    {u'confidence': 0.8764819,
     u'level': u'SHOT_LEVEL',
     u'segment': {u'endTimeOffset': u'23533281',
      u'startTimeOffset': u'21733318'}},
    {u'confidence': 0.82209057,
     u'level': u'SHOT_LEVEL',
     u'segment': {u'endTimeOffset': u'25666689',
      u'startTimeOffset': u'23599970'}}]},
  {u'description': u'Animation',
   u'languageCode': 

In [21]:
len(video_response['response']['annotationResults'][0]['labelAnnotations'])

23

In [31]:
video_response['response']['annotationResults'][0]['labelAnnotations'][0]['locations']

[{u'confidence': 0.8764819,
  u'level': u'VIDEO_LEVEL',
  u'segment': {u'endTimeOffset': u'-1', u'startTimeOffset': u'-1'}},
 {u'confidence': 0.89243287,
  u'level': u'SHOT_LEVEL',
  u'segment': {u'endTimeOffset': u'8333309'}},
 {u'confidence': 0.4363379,
  u'level': u'SHOT_LEVEL',
  u'segment': {u'endTimeOffset': u'12933314', u'startTimeOffset': u'8399998'}},
 {u'confidence': 0.7916033,
  u'level': u'SHOT_LEVEL',
  u'segment': {u'endTimeOffset': u'15999984',
   u'startTimeOffset': u'13000003'}},
 {u'confidence': 0.8764819,
  u'level': u'SHOT_LEVEL',
  u'segment': {u'endTimeOffset': u'23533281',
   u'startTimeOffset': u'21733318'}},
 {u'confidence': 0.82209057,
  u'level': u'SHOT_LEVEL',
  u'segment': {u'endTimeOffset': u'25666689',
   u'startTimeOffset': u'23599970'}}]

In [32]:
for i in range(len(video_response['response']['annotationResults'][0]['labelAnnotations'])):
    print('{}'.format(
              video_response['response']['annotationResults'][0]['labelAnnotations'][i]['description']
#             , video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations']
        ))

Animal
Animation
Bud
Flora
Flower
Flowering plant
Giant panda
Grass
Grassland
Hare
Leaf
Meadow
Nature
Pasture
Petal
Plant
Plant stem
Rabbit
Rose
Sky
Sunlight
Tree
Wildlife


In [19]:
video_response['response']['annotationResults'][0]['labelAnnotations']

[{u'description': u'Animal',
  u'languageCode': u'en-us',
  u'locations': [{u'confidence': 0.8764819,
    u'level': u'VIDEO_LEVEL',
    u'segment': {u'endTimeOffset': u'-1', u'startTimeOffset': u'-1'}},
   {u'confidence': 0.89243287,
    u'level': u'SHOT_LEVEL',
    u'segment': {u'endTimeOffset': u'8333309'}},
   {u'confidence': 0.4363379,
    u'level': u'SHOT_LEVEL',
    u'segment': {u'endTimeOffset': u'12933314',
     u'startTimeOffset': u'8399998'}},
   {u'confidence': 0.7916033,
    u'level': u'SHOT_LEVEL',
    u'segment': {u'endTimeOffset': u'15999984',
     u'startTimeOffset': u'13000003'}},
   {u'confidence': 0.8764819,
    u'level': u'SHOT_LEVEL',
    u'segment': {u'endTimeOffset': u'23533281',
     u'startTimeOffset': u'21733318'}},
   {u'confidence': 0.82209057,
    u'level': u'SHOT_LEVEL',
    u'segment': {u'endTimeOffset': u'25666689',
     u'startTimeOffset': u'23599970'}}]},
 {u'description': u'Animation',
  u'languageCode': u'en-us',
  u'locations': [{u'confidence': 0.60

In [20]:
video_response['response']['annotationResults'][0]['shotAnnotations']

[{u'endTimeOffset': u'8333309'},
 {u'endTimeOffset': u'12933314', u'startTimeOffset': u'8399998'},
 {u'endTimeOffset': u'15999984', u'startTimeOffset': u'13000003'},
 {u'endTimeOffset': u'21666629', u'startTimeOffset': u'16066673'},
 {u'endTimeOffset': u'23533281', u'startTimeOffset': u'21733318'},
 {u'endTimeOffset': u'25666689', u'startTimeOffset': u'23599970'},
 {u'endTimeOffset': u'26733329', u'startTimeOffset': u'25733378'}]

In [ ]:
for name, age in list.iteritems():
    if age == search_age:
        print name

In [ ]:
for name, age in list.iteritems():
    if age == search_age:
        print name

In [35]:
search_label = 'Animal'

In [36]:
for i in range(len(video_response['response']['annotationResults'][0]['labelAnnotations'])):
    if video_response['response']['annotationResults'][0]['labelAnnotations'][i]['description'] == search_label:
        print(video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'])
        

[{u'confidence': 0.8764819, u'segment': {u'endTimeOffset': u'-1', u'startTimeOffset': u'-1'}, u'level': u'VIDEO_LEVEL'}, {u'confidence': 0.89243287, u'segment': {u'endTimeOffset': u'8333309'}, u'level': u'SHOT_LEVEL'}, {u'confidence': 0.4363379, u'segment': {u'endTimeOffset': u'12933314', u'startTimeOffset': u'8399998'}, u'level': u'SHOT_LEVEL'}, {u'confidence': 0.7916033, u'segment': {u'endTimeOffset': u'15999984', u'startTimeOffset': u'13000003'}, u'level': u'SHOT_LEVEL'}, {u'confidence': 0.8764819, u'segment': {u'endTimeOffset': u'23533281', u'startTimeOffset': u'21733318'}, u'level': u'SHOT_LEVEL'}, {u'confidence': 0.82209057, u'segment': {u'endTimeOffset': u'25666689', u'startTimeOffset': u'23599970'}, u'level': u'SHOT_LEVEL'}]


In [37]:
search_label = 'Rabbit'

In [38]:
for i in range(len(video_response['response']['annotationResults'][0]['labelAnnotations'])):
    if video_response['response']['annotationResults'][0]['labelAnnotations'][i]['description'] == search_label:
        print(video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'])
        

[{u'confidence': 0.9933143, u'segment': {u'endTimeOffset': u'15999984', u'startTimeOffset': u'13000003'}, u'level': u'SHOT_LEVEL'}]


In [39]:
search_label = 'Sam Gu'

In [40]:
for i in range(len(video_response['response']['annotationResults'][0]['labelAnnotations'])):
    if video_response['response']['annotationResults'][0]['labelAnnotations'][i]['description'] == search_label:
        print(video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'])
        

### Format Video Analysis Results

In [ ]:
video_response = KudosData_VIDEO_DETECTION(encode_media(video_file))

In [ ]:
video_response

# WIP 2

### * 识别图片消息中的物体名字 (Recognize objects in image) 
    [1] 物体名 (General Object)

In [ ]:
# Running Vision API
# 'LABEL_DETECTION'
def KudosData_LABEL_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\n[ ' + API_type + u' 物体识别 ]\n'
    # 'LABEL_DETECTION'
    if responses['responses'][0] != {}:
        for i in range(len(responses['responses'][0]['labelAnnotations'])):
            image_analysis_reply += responses['responses'][0]['labelAnnotations'][i]['description'] \
            + '\n( confidence ' +  str(responses['responses'][0]['labelAnnotations'][i]['score']) + ' )\n'
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
        
    return image_analysis_reply

### * 识别图片消息中的物体名字 (Recognize objects in image) 
    [2] 地标名 (Landmark Object)

In [ ]:
# Running Vision API
# 'LANDMARK_DETECTION'
def KudosData_LANDMARK_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\n[ ' + API_type + u' 地标识别 ]\n'
    # 'LANDMARK_DETECTION'
    if responses['responses'][0] != {}:
        for i in range(len(responses['responses'][0]['landmarkAnnotations'])):
            image_analysis_reply += responses['responses'][0]['landmarkAnnotations'][i]['description'] \
            + '\n( confidence ' +  str(responses['responses'][0]['landmarkAnnotations'][i]['score']) + ' )\n'
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
        
    return image_analysis_reply

### * 识别图片消息中的物体名字 (Recognize objects in image) 
    [3] 商标名 (Logo Object)

In [ ]:
# Running Vision API
# 'LOGO_DETECTION'
def KudosData_LOGO_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\n[ ' + API_type + u' 商标识别 ]\n'
    # 'LOGO_DETECTION'
    if responses['responses'][0] != {}:
        for i in range(len(responses['responses'][0]['logoAnnotations'])):
            image_analysis_reply += responses['responses'][0]['logoAnnotations'][i]['description'] \
            + '\n( confidence ' +  str(responses['responses'][0]['logoAnnotations'][i]['score']) + ' )\n'
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
        
    return image_analysis_reply

### * 识别图片消息中的文字 (OCR: Extract text from image)

In [ ]:
# Running Vision API
# 'TEXT_DETECTION'
def KudosData_TEXT_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\n[ ' + API_type + u' 文字提取 ]\n'
    # 'TEXT_DETECTION'
    if responses['responses'][0] != {}:
        image_analysis_reply += u'----- Start Original Text -----\n'
        image_analysis_reply += u'( Original Language 原文: ' + responses['responses'][0]['textAnnotations'][0]['locale'] \
        + ' )\n'        
        image_analysis_reply += responses['responses'][0]['textAnnotations'][0]['description'] + '----- End Original Text -----\n'

        ##############################################################################################################
        #                                        translation of detected text                                        #
        ##############################################################################################################
        parm_translation_origin_language = responses['responses'][0]['textAnnotations'][0]['locale']
        # Call translation if parm_translation_origin_language is not parm_translation_target_language
        if parm_translation_origin_language != parm_translation_target_language:
            inputs=[responses['responses'][0]['textAnnotations'][0]['description']] # TEXT_DETECTION OCR results only
            outputs = service.translations().list(source=parm_translation_origin_language, 
                                                  target=parm_translation_target_language, q=inputs).execute()
            image_analysis_reply += u'\n----- Start Translation -----\n'
            image_analysis_reply += u'( Target Language 译文: ' + parm_translation_target_language + ' )\n'
            image_analysis_reply += outputs['translations'][0]['translatedText'] + '\n' + '----- End Translation -----\n'
            print('Compeleted: Translation    API ...')
        ##############################################################################################################
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
        
    return image_analysis_reply

### * 识别人脸 (Recognize human face)
### * 基于人脸的表情来识别喜怒哀乐等情绪 (Identify sentiment and emotion from human face)

In [ ]:
# Running Vision API
# 'FACE_DETECTION'
def KudosData_FACE_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\n[ ' + API_type + u' 面部表情 ]\n'
    # 'FACE_DETECTION'
    if responses['responses'][0] != {}:
        for i in range(len(responses['responses'][0]['faceAnnotations'])):
            image_analysis_reply += u'----- No.' + str(i+1) + ' Face -----\n'
            
            image_analysis_reply += u'>>> Joy 喜悦: \n' \
            + responses['responses'][0]['faceAnnotations'][i][u'joyLikelihood'] + '\n'
            
            image_analysis_reply += u'>>> Anger 愤怒: \n' \
            + responses['responses'][0]['faceAnnotations'][i][u'angerLikelihood'] + '\n'
            
            image_analysis_reply += u'>>> Sorrow 悲伤: \n' \
            + responses['responses'][0]['faceAnnotations'][i][u'sorrowLikelihood'] + '\n'
            
            image_analysis_reply += u'>>> Surprise 惊奇: \n' \
            + responses['responses'][0]['faceAnnotations'][i][u'surpriseLikelihood'] + '\n'
            
            image_analysis_reply += u'>>> Headwear 头饰: \n' \
            + responses['responses'][0]['faceAnnotations'][i][u'headwearLikelihood'] + '\n'
            
            image_analysis_reply += u'>>> Blurred 模糊: \n' \
            + responses['responses'][0]['faceAnnotations'][i][u'blurredLikelihood'] + '\n'
            
            image_analysis_reply += u'>>> UnderExposed 欠曝光: \n' \
            + responses['responses'][0]['faceAnnotations'][i][u'underExposedLikelihood'] + '\n'
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
            
    return image_analysis_reply

### * 不良内容识别 (Explicit Content Detection)

Detect explicit content like adult content or violent content within an image.

In [ ]:
# Running Vision API
# 'SAFE_SEARCH_DETECTION'
def KudosData_SAFE_SEARCH_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\n[ ' + API_type + u' 不良内容 ]\n'
    # 'SAFE_SEARCH_DETECTION'
    if responses['responses'][0] != {}:
        image_analysis_reply += u'>>> Adult 成人: \n' + responses['responses'][0]['safeSearchAnnotation'][u'adult'] + '\n'
        image_analysis_reply += u'>>> Violence 暴力: \n' + responses['responses'][0]['safeSearchAnnotation'][u'violence'] + '\n'
        image_analysis_reply += u'>>> Spoof 欺骗: \n' + responses['responses'][0]['safeSearchAnnotation'][u'spoof'] + '\n'
        image_analysis_reply += u'>>> Medical 医疗: \n' + responses['responses'][0]['safeSearchAnnotation'][u'medical'] + '\n'
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
    return image_analysis_reply

### 用微信App扫QR码图片来自动登录

In [ ]:
itchat.auto_login(hotReload=True) # hotReload=True: 退出程序后暂存登陆状态。即使程序关闭，一定时间内重新开启也可以不用重新扫码。
# itchat.auto_login(enableCmdQR=-2) # enableCmdQR=-2: 命令行显示QR图片

In [ ]:
# @itchat.msg_register([PICTURE], isGroupChat=True)
@itchat.msg_register([PICTURE])
def download_files(msg):
    parm_translation_origin_language = 'zh' # will be overwriten by TEXT_DETECTION
    msg.download(msg.fileName)
    print('\nDownloaded image file name is: %s' % msg['FileName'])
    image_base64 = encode_image(msg['FileName'])
    
    ##############################################################################################################
    #                                          call image analysis APIs                                          #
    ##############################################################################################################
    
    image_analysis_reply = u'[ Image Analysis 图像分析结果 ]\n'

    # 1. LABEL_DETECTION:
    image_analysis_reply += KudosData_LABEL_DETECTION(image_base64, 'LABEL_DETECTION', parm_image_maxResults)
    # 2. LANDMARK_DETECTION:
    image_analysis_reply += KudosData_LANDMARK_DETECTION(image_base64, 'LANDMARK_DETECTION', parm_image_maxResults)
    # 3. LOGO_DETECTION:
    image_analysis_reply += KudosData_LOGO_DETECTION(image_base64, 'LOGO_DETECTION', parm_image_maxResults)
    # 4. TEXT_DETECTION:
    image_analysis_reply += KudosData_TEXT_DETECTION(image_base64, 'TEXT_DETECTION', parm_image_maxResults)
    # 5. FACE_DETECTION:
    image_analysis_reply += KudosData_FACE_DETECTION(image_base64, 'FACE_DETECTION', parm_image_maxResults)
    # 6. SAFE_SEARCH_DETECTION:
    image_analysis_reply += KudosData_SAFE_SEARCH_DETECTION(image_base64, 'SAFE_SEARCH_DETECTION', parm_image_maxResults)

    print('Compeleted: Image Analysis API ...')
    
    return image_analysis_reply

In [ ]:
itchat.run()

In [ ]:
# interupt kernel, then logout
itchat.logout() # 安全退出

### 恭喜您！已经完成了：
### 第二课：图像识别和处理
### Lesson 2: Image Recognition & Processing

* 识别图片消息中的物体名字 (Recognize objects in image)
        [1] 物体名 (General Object)
        [2] 地标名 (Landmark Object)
        [3] 商标名 (Logo Object)

* 识别图片消息中的文字 (OCR: Extract text from image)
        包含简单文本翻译 (Call text translation API)
        
* 识别人脸 (Recognize human face)
        基于人脸的表情来识别喜怒哀乐等情绪 (Identify sentiment and emotion from human face)

* 不良内容识别 (Explicit Content Detection)


### 下一课是:
### 第三课：自然语言处理：语音合成和识别
### Lesson 3: Natural Language Processing 1
* 消息文字转成语音 (Speech synthesis: text to voice)
* 语音转换成消息文字 (Speech recognition: voice to text)
* 消息文字的多语言互译 (Text based language translation)

<img src='http://www.kudosdata.com/wp-content/uploads/2016/11/cropped-KudosLogo1.png' width=30% style="float: right;">
<img src='reference/WeChat_SamGu_QR.png' width=10% style="float: left;">

